In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TesteVSCode").getOrCreate()
print("Conectado ao Spark no Docker. Versão:", spark.version)

Conectado ao Spark no Docker. Versão: 3.5.0
